# SIT744 Practical 4: Second look at TensorFlow and Keras


*Dr Wei Luo*




<div class="alert alert-info">
We suggest that you run this notebook using Google Colab.
</div>

## Pre-practical readings

- [TensorFlow tensors](https://www.tensorflow.org/guide/tensor)
- [TensorFlow Variables](https://www.tensorflow.org/guide/variable)



## Task 1. Low-level tensor manipulation in TensorFlow

TensorFlow APIs cover three deep-learning components:

- Tensors, including variables for keeping layer weights
- Tensor operations, including tensor multiplication,  addition, and the activation functions
- Backpropagation, implemented through the GradientTape object

In this task, we will learn how to use these TensorFlow components. 

### Task 1.1 Tensors and tensor operations



#### tf.Tensor vs tensors

We mentioned that tensors (with lower-case t) are like NumPy arrays. However, [`tf.Tensor`](https://www.tensorflow.org/api_docs/python/tf/Tensor) (with upper-case T) is different. A tf.Tensor defines a function (or computation) that, when called, produces a tensor. Such a function is called an **Operator** in the TensorFlow nomenclature.

Therefore, a TensorFlow program forms a computation graph that chains a collection of tf.Tensor's together. You can think that the program  itself contains no data. When we invoke the program, data (tensors) are generated or passed in and flow through operators in the program. But how can we store the model parameters and train the model parameters?



#### Constant tensors and Variables

Assuming that we separate data from computation. Then `tf.constant` and `tf.Variable` are the two main `tf.Tensor` that directly deal with data. Data in `tf.constant` is immutable and data in `tf.Variable` can be changed.

Constant tensors return the same data at every invocation of the computation graph.


In [0]:

import tensorflow as tf

a = tf.constant([[1, 2],
                 [3, 4]])
print(a)


As you can see that `tf.constant` returns a tensor. There are some other functions that return constant tensors.

In [0]:
x = tf.ones(shape=(2, 2))
print(x)
x = tf.zeros(shape=(2, 2))
print(x)

A particularly important class of functions are those used for generating random initial weights in a network. They produced constant tensors as the initial values for variables.  

In [0]:
## Normal initialiser
w_init = tf.random_normal_initializer(
    mean=0.0, stddev=0.05, seed=None
)

initial_value=w_init(shape=(2, 2),
                     dtype='float32')

print(initial_value)


## Uniform initialiser
w_init = tf.random_uniform_initializer(
    minval = -0.05, maxval = 0.05, seed=None
)

initial_value=w_init(shape=(2, 2),
                     dtype='float32')

print(initial_value)



**exercise** Try to assign a new value to a constant tensor. Can you do that?

In comparison to constants, variables can be assigned a different value. They are required to represent trainable network/layer weights

In [0]:
w = tf.Variable(initial_value=initial_value,
                         trainable=True)
print(w)

You can use the `assign` function to change the value in a variable.

In [0]:
print(w[0,0])


w[0,0].assign(0)
print(w[0,0])

### Task 1.2 Math operations in TensorFlow

The transformation of tensors are achieved by matrix multiplications, additions, reshaping, and activation functions.

In [0]:
a = tf.ones((2, 2))
print(f'a: {a}')
b = tf.square(a)
print(f'b: {b}')
c = tf.sqrt(a)
print(f'c: {c}')
d = b + c
print(f'd = b + c: {d}')
e = tf.matmul(a, b)
print(f'e = tf.matmul(a, b): {e}')
e *= d
print(f'e *= d: {e}')

### Task 1.3 Performing differentiation with tensor operations

Tensor operations come with the ability to perform automatic differentiation.

In [0]:
x = tf.Variable(initial_value= tf.ones((2, 2)))

with tf.GradientTape() as tape:
   y = tf.square(x)
dy_dx = tape.gradient(y, x)
print(dy_dx)

with tf.GradientTape() as paper:
   z = tf.sqrt(x)

dz_dx = paper.gradient(z, [x])


print(dz_dx)

**exercise** Modify the code above to compute the gradient of the exponential function $y=e^{x}$.

**question** Can you call `tape.gradient` twice?


## Task 2  Reimplementing a Keras model in TensorFlow

Last week, we used a Keras model to run through the MNIST example. In this practical, we learn how to reimplement the model without using Keras. This will deepen your understanding of some key concepts.



### Task 2.1 A simple Dense class

We know that a dense layer is essentially performing an affine transformation followed by an activation function.

> output = activation(dot(W, input) + b)



We can define a class for network layers.


In [0]:
class NaiveDense:

    def __init__(self, units, input_dim, activation):
        self.activation = activation

        W_init = tf.random_normal_initializer()
        self.W = tf.Variable(initial_value=W_init(shape=(input_dim, units),
                                              dtype='float32'),
                         trainable=True)

        b_init = tf.zeros_initializer()
        self.b = tf.Variable(initial_value=b_init(shape=(units,),
                                              dtype='float32'),
                         trainable=True)
        
    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b)

    @property
    def weights(self):
        return [self.W, self.b]

Let's try to pass a tensor through the layer.

In [0]:
relu_layer = NaiveDense(units=10, input_dim=2, activation = tf.nn.relu)

x = tf.ones((2, 2))
y = relu_layer(x)
print(y)



You can compare this with the original implementation from Keras. The difference in values is due to random initialisation.

In [0]:
keras_layer = tf.keras.layers.Dense(units=10, activation = tf.nn.relu)
y = keras_layer(x)
print(y)

**question** Do you see negative values in the output? Why? Are we using the 10 output units effectively?

### Task 2.2 A simple Sequential class

Once we have defined some layers, we can chain them together. Let's define a class similar to the Sequential Model in Keras.







In [0]:
class NaiveSequential:

    def __init__(self, layers):
        self.layers = layers

    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
           x = layer(x)
        return x

    @property
    def weights(self):
       weights = []
       for layer in self.layers:
           weights += layer.weights
       return weights

It takes a list of layers and returns a model.

In [0]:
model = NaiveSequential([
    NaiveDense(units=512, input_dim=28 * 28, activation=tf.nn.relu),
    NaiveDense(units=10, input_dim=512,  activation=tf.nn.softmax)
])


Let's try to feed the model two identical "images".


In [0]:
model(tf.ones((2, 28 * 28)))

### Task 2.3 A batch generator

To run stochastic gradient-descent, we need to feed the model with mini-batches of the input data. Later on, we will learn how to build TensorFlow input pipelines with `tf.data`. Here we will create a simple iterator for retrieving training batches.

In [0]:
class BatchGenerator:

    def __init__(self, images, labels, batch_size=128):
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size

    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

Let's try it on the MNIST data.

In [0]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

batch_generator = BatchGenerator(train_images, train_labels)
x, y = batch_generator.next()
print(f'x: {x}')
print(f'y: {y}')

## Task 3 Training the model

As mentioned in the lecture. training a neural network involves a loop with the following steps:

1. Compute the predictions of the examples in the batch
2. Compute the loss value for these predictions given the actual labels
3. Compute the gradient of the loss with regard to the model’s weights
4. Move the weights by a small amount in the direction opposite to the gradient

These four steps comprise one **training step**.

In [0]:
learning_rate = 1e-3

def update_weights(gradients, weights):
    for g, w in zip(gradients, model.weights):
        w.assign_sub(g * learning_rate)

def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
      predictions = model(images_batch) ## 1. Compute the predictions of the examples in the batch 
      per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
          labels_batch, predictions) 
      average_loss = tf.reduce_mean(per_sample_losses) ## 2. Compute the loss value for these predictions given the actual labels
    gradients = tape.gradient(average_loss, model.weights) ## 3. Compute the gradient of the loss with regard to the model’s weights
    update_weights(gradients, model.weights) ## 4. Move the weights by a small amount in the direction opposite to the gradient
    return average_loss

Let's test-run it with a training batch.

In [0]:
one_training_step(model, x, y)

Knowing that it is working, we can add a for-loop. Actually, we will use two nested for-loops: the outer for-loop to keep track of the number of epochs and the inner for-loop to iterate through the whole training data (multiple mini-batches). 



In [0]:
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
      print('Epoch %d' % epoch_counter)
      batch_generator = BatchGenerator(images, labels)
      for batch_counter in range(len(images) // batch_size):
          images_batch, labels_batch = batch_generator.next()
          loss = one_training_step(model, images_batch, labels_batch)
          if batch_counter % 100 == 0:
              print('loss at batch %d: %.2f' % (batch_counter, loss))

Now we are ready to train the model. Specify 5 epochs.

In [0]:
fit(model, train_images, train_labels, epochs=50, batch_size=2)

**exercise** 

1. Write a program to evaluate the accuracy of the model and evaluate the accuracy on both training and test datasets.

2. Modify the code above so that you collect the gradients at each layer and each epoch.


## Additional resources

- [Tensorflow, The Confusing Parts (1)](https://jacobbuckman.com/2018-06-25-tensorflow-the-confusing-parts-1/). Don't worry if some parts are still "confusing" after reading this.